In [2]:
import os
import glob
import pandas as pd
import itertools

import Bio
from Bio import Entrez, SeqIO
from Bio import AlignIO

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
#path to bam in csv format
path_to_bam_file = '../Results_fragseq_rep1/Fraq_Seq_S16_S16_R1_001_R2_combi_ref_proper_20_40.csv'
#path to output df
df_align = '../Results_fragseq_rep1/S16_S16_20_40_align_df.csv'



+ open a converted to csv bam table

In [20]:
bbbb = pd.read_csv(path_to_bam_file,  sep =' ') #, header=None)
bbbb.shape[0]

415524

In [6]:
bbbb.head(100)

,qname,flag,rname,strand,pos,qwidth,mapq,cigar,mrnm,mpos,isize,seq,qual
1,A01492:10:HNH2FDMXX:1:1101:10086:11569,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
2,A01492:10:HNH2FDMXX:1:1102:29948:28338,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
3,A01492:10:HNH2FDMXX:1:1115:22092:3364,163,30,+,1,28,60,28M,30,1,28,TGTCAGTGACTGCGCCAAAACATCTTCG,FFFFFFFFFFFFFFFFFFF:FFFFFFFF
4,A01492:10:HNH2FDMXX:1:1125:21016:24549,99,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
5,A01492:10:HNH2FDMXX:1:1136:28103:35383,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,A01492:10:HNH2FDMXX:2:1122:3658:34695,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
97,A01492:10:HNH2FDMXX:2:1132:16975:22185,99,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
98,A01492:10:HNH2FDMXX:2:1133:13440:24017,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
99,A01492:10:HNH2FDMXX:2:1137:29821:26365,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF


+ rename columns and moderate them

In [7]:
bbbb.rename(columns = {'qname' : 'ID', 'mrnm' : 'DNA', 'mpos' : 'left', 'isize' : 'width'}, inplace = True)
bbbb = bbbb.drop(columns = ['pos','qwidth', 'mapq', 'seq','qual'], axis = 1)


In [8]:
bbbb = bbbb.drop(columns = ['strand'], axis = 1)

In [9]:
bbbb.shape

(415524, 7)

In [10]:
bbbb

,ID,flag,rname,cigar,DNA,left,width
1,A01492:10:HNH2FDMXX:1:1101:10086:11569,163,30,30M,30,1,30
2,A01492:10:HNH2FDMXX:1:1102:29948:28338,163,30,30M,30,1,30
3,A01492:10:HNH2FDMXX:1:1115:22092:3364,163,30,28M,30,1,28
4,A01492:10:HNH2FDMXX:1:1125:21016:24549,99,30,30M,30,1,30
5,A01492:10:HNH2FDMXX:1:1136:28103:35383,163,30,30M,30,1,30
...,...,...,...,...,...,...,...
415520,A01492:10:HNH2FDMXX:2:1110:3956:14826,83,pCDF_Ec.cas1/2,21M,pCDF_Ec.cas1/2,4714,-21
415521,A01492:10:HNH2FDMXX:1:2347:29559:3176,163,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20
415522,A01492:10:HNH2FDMXX:2:2145:18376:12305,99,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20
415523,A01492:10:HNH2FDMXX:1:2347:29559:3176,83,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,-20


In [11]:
def module_val(s):
    return abs(s)

def plus_minus(s):
    if s == 99 or s == 147:
        return "dir"
    elif s == 83 or s == 163:
        return "rev"

In [12]:
bbbb['flag'] = bbbb['flag'].apply(plus_minus)
bbbb['width'] = bbbb['width'].apply(module_val)
bbbb['right'] = bbbb['left'] + bbbb['width'] - 1
bbbb

,ID,flag,rname,cigar,DNA,left,width,right
1,A01492:10:HNH2FDMXX:1:1101:10086:11569,rev,30,30M,30,1,30,30
2,A01492:10:HNH2FDMXX:1:1102:29948:28338,rev,30,30M,30,1,30,30
3,A01492:10:HNH2FDMXX:1:1115:22092:3364,rev,30,28M,30,1,28,28
4,A01492:10:HNH2FDMXX:1:1125:21016:24549,dir,30,30M,30,1,30,30
5,A01492:10:HNH2FDMXX:1:1136:28103:35383,rev,30,30M,30,1,30,30
...,...,...,...,...,...,...,...,...
415520,A01492:10:HNH2FDMXX:2:1110:3956:14826,rev,pCDF_Ec.cas1/2,21M,pCDF_Ec.cas1/2,4714,21,4734
415521,A01492:10:HNH2FDMXX:1:2347:29559:3176,rev,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20,4734
415522,A01492:10:HNH2FDMXX:2:2145:18376:12305,dir,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20,4734
415523,A01492:10:HNH2FDMXX:1:2347:29559:3176,rev,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20,4734


In [13]:
bbbb = bbbb.drop(columns = ['rname', 'cigar'], axis = 1)
bbbb.rename(columns = {'width' : 'isize'}, inplace = True)
bbbb['width'] = bbbb['isize']
bbbb = bbbb.drop(columns = ['isize'], axis = 1)
bbbb.rename(columns = {'flag' : 'strand'}, inplace = True)

bbbb

,ID,strand,DNA,left,right,width
1,A01492:10:HNH2FDMXX:1:1101:10086:11569,rev,30,1,30,30
2,A01492:10:HNH2FDMXX:1:1102:29948:28338,rev,30,1,30,30
3,A01492:10:HNH2FDMXX:1:1115:22092:3364,rev,30,1,28,28
4,A01492:10:HNH2FDMXX:1:1125:21016:24549,dir,30,1,30,30
5,A01492:10:HNH2FDMXX:1:1136:28103:35383,rev,30,1,30,30
...,...,...,...,...,...,...
415520,A01492:10:HNH2FDMXX:2:1110:3956:14826,rev,pCDF_Ec.cas1/2,4714,4734,21
415521,A01492:10:HNH2FDMXX:1:2347:29559:3176,rev,pCDF_Ec.cas1/2,4715,4734,20
415522,A01492:10:HNH2FDMXX:2:2145:18376:12305,dir,pCDF_Ec.cas1/2,4715,4734,20
415523,A01492:10:HNH2FDMXX:1:2347:29559:3176,rev,pCDF_Ec.cas1/2,4715,4734,20


+ save the dataframe or before that choose the reference (see below)

In [14]:
#df_align = '/Users/unnost/workwork/prespacers_task/Results_fragseq/Oct14_align_df.csv'
bbbb.to_csv(df_align, index=False)

+ here we can possibly choose a particular reference

In [15]:
bbbb = bbbb.loc[bbbb['DNA'] == "BL21_AI_CP047231"]

In [16]:
bbbb

,ID,strand,DNA,left,right,width
28509,A01492:10:HNH2FDMXX:1:2160:30092:22013,rev,BL21_AI_CP047231,1,34,34
28510,A01492:10:HNH2FDMXX:1:2374:3477:11490,dir,BL21_AI_CP047231,1,24,24
28511,A01492:10:HNH2FDMXX:1:2374:1262:13980,dir,BL21_AI_CP047231,1,24,24
28512,A01492:10:HNH2FDMXX:1:2443:12436:25786,dir,BL21_AI_CP047231,1,37,37
28513,A01492:10:HNH2FDMXX:1:2450:19379:11036,rev,BL21_AI_CP047231,1,24,24
...,...,...,...,...,...,...
377070,A01492:10:HNH2FDMXX:2:2141:25310:34209,dir,BL21_AI_CP047231,4530535,4530564,30
377071,A01492:10:HNH2FDMXX:1:2470:1994:9142,dir,BL21_AI_CP047231,4530535,4530564,30
377072,A01492:10:HNH2FDMXX:2:2141:25310:34209,dir,BL21_AI_CP047231,4530535,4530564,30
377073,A01492:10:HNH2FDMXX:1:1134:5981:20149,dir,BL21_AI_CP047231,4530537,4530564,28


In [17]:
bbbb.shape

(348566, 6)